## Zipcode measure Feature Engineering

In [1]:
import pandas as pd

In [2]:
import seaborn as sns; sns.set_style("dark")

#### Loading in data

In [3]:
%cd ..

/workspace/asurion_f22_muby


In [4]:
df = pd.read_csv("fe_data/total_encoded_tabular.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1211571 entries, 0 to 1211570
Data columns (total 43 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   id                          1211571 non-null  object 
 1   quality_sale                1211571 non-null  int64  
 2   accepted_flg                1211571 non-null  int64  
 3   seven_day_churn             105425 non-null   float64
 4   thirty_day_churn            105425 non-null   float64
 5   ninety_day_churn            105425 non-null   float64
 6   clndr_dt_cst                1211571 non-null  object 
 7   start_dt                    1211571 non-null  object 
 8   lines                       1211571 non-null  int64  
 9   previous_tech_support_call  1211571 non-null  int64  
 10  account_tenure              1211571 non-null  int64  
 11  past_yr_claim_count         1211571 non-null  int64  
 12  num_decline                 1211571 non-null  int64  
 1

In [6]:
df.head()

,id,quality_sale,accepted_flg,seven_day_churn,thirty_day_churn,ninety_day_churn,clndr_dt_cst,start_dt,lines,previous_tech_support_call,...,county,state,state_code,latitude,longitude,month,hh_income_code,day_of_week,time_of_day,zipcode_avg_sales
0,c90ce3eaafffd07a32cb9cf4d153aef6f524b29aeef447...,0,0,NaN,NaN,NaN,2022-03-21 00:00:00.000,2022-03-21 20:44:17.405,2,15,...,Scottsdale,Arizona,AZ,33.7669,-111.9182,march,1,Monday,20,0.041667
1,91aa94d54079b20e2a18b09c070d0304b3df9c7272e9fc...,0,0,NaN,NaN,NaN,2022-03-03 00:00:00.000,2022-03-03 21:16:27.713,1,10,...,Salisbury,Maryland,MD,38.3824,-75.6336,march,3,Thursday,21,0.105590
2,eb269842f94437bfd7e3a5a1f30d7b9a84a842afd8a153...,0,0,NaN,NaN,NaN,2022-03-16 00:00:00.000,2022-03-16 13:59:15.707,5,30,...,Valencia,California,CA,34.4466,-118.5374,march,1,Wednesday,13,0.058824
3,fc05f45119a8a4b830a3463bfc8a7113b2807c88e7d2da...,0,0,NaN,NaN,NaN,2022-03-01 00:00:00.000,2022-03-01 12:33:55.917,5,1,...,Kaneohe,Hawaii,HI,21.4228,-157.8115,march,1,Tuesday,12,0.084034
4,68943556bf63277816d843ffb129e23ffe60bc5b982603...,1,1,0.0,0.0,0.0,2022-03-01 00:00:00.000,2022-03-01 12:22:15.780,2,4,...,Conroe,Texas,TX,30.3125,-95.4527,march,4,Tuesday,12,0.151163


In [7]:
zipcode_df = df.groupby(["zipcode","quality_sale"], as_index = False).size()
zipcode_df = zipcode_df.pivot_table(index = "zipcode", values = "size", columns = "quality_sale")
zipcode_df = zipcode_df.fillna(0)

In [8]:
zipcode_df["mean_sales"] = zipcode_df.loc[:,1]/(zipcode_df.loc[:,0] + zipcode_df.loc[:,1])
zipcode_df["zipcode"] = zipcode_df.index

In [9]:
zipcode_df = zipcode_df.reset_index(drop = True)

In [10]:
zipcode_df["total_sales"] = zipcode_df[0] + zipcode_df[1]

In [11]:
zipcode_df = zipcode_df.rename_axis(None, axis=1)

In [12]:
#zipcode_df = zipcode_df.drop([0,1], axis = 1)

In [13]:
new_data = df.merge(zipcode_df, how = "left", on = "zipcode")

In [14]:
new_data = new_data.rename(columns = {0:"zipcode_neg_sales",1:"zipcode_pos_sales","total_sales":"zipcode_total_sales"})

In [15]:
new_data = new_data.drop("mean_sales", axis = 1)

In [16]:
new_data.columns

Index(['id', 'quality_sale', 'accepted_flg', 'seven_day_churn',
       'thirty_day_churn', 'ninety_day_churn', 'clndr_dt_cst', 'start_dt',
       'lines', 'previous_tech_support_call', 'account_tenure',
       'past_yr_claim_count', 'num_decline', 'exp', 'product_type', 'pmake',
       'claritas_hh_count', 'technodoption_class', 'hh_income', 'age_class',
       'hh_education', 'hh_composition', 'hh_tenure', 'urbanicity',
       'median_income', 'total_population', 'perc_female', 'perc_under18',
       'perc_over65', 'perc_white', 'perc_black', 'population_density',
       'zipcode', 'county', 'state', 'state_code', 'latitude', 'longitude',
       'month', 'hh_income_code', 'day_of_week', 'time_of_day',
       'zipcode_avg_sales', 'zipcode_neg_sales', 'zipcode_pos_sales',
       'zipcode_total_sales'],
      dtype='object')

In [17]:
new_data = new_data[['id','quality_sale', 'accepted_flg', 'seven_day_churn', 'thirty_day_churn',
                    'ninety_day_churn', 'clndr_dt_cst', 'start_dt', 'lines',
                    'previous_tech_support_call', 'account_tenure', 'past_yr_claim_count',
                    'num_decline', 'exp', 'product_type', 'pmake', 'claritas_hh_count',
                    'technodoption_class', 'hh_income', 'age_class', 'hh_education',
                    'hh_composition', 'hh_tenure', 'urbanicity', 'median_income',
                    'total_population', 'perc_female', 'perc_under18', 'perc_over65',
                    'perc_white', 'perc_black', 'population_density', 'zipcode', 'county', 
                    'state', 'state_code', 'latitude', 'longitude',
                    'month', 'hh_income_code', 'day_of_week', 'time_of_day',
                    'zipcode_avg_sales',  'zipcode_neg_sales', 'zipcode_pos_sales', 'zipcode_total_sales']]
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1211571 entries, 0 to 1211570
Data columns (total 46 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   id                          1211571 non-null  object 
 1   quality_sale                1211571 non-null  int64  
 2   accepted_flg                1211571 non-null  int64  
 3   seven_day_churn             105425 non-null   float64
 4   thirty_day_churn            105425 non-null   float64
 5   ninety_day_churn            105425 non-null   float64
 6   clndr_dt_cst                1211571 non-null  object 
 7   start_dt                    1211571 non-null  object 
 8   lines                       1211571 non-null  int64  
 9   previous_tech_support_call  1211571 non-null  int64  
 10  account_tenure              1211571 non-null  int64  
 11  past_yr_claim_count         1211571 non-null  int64  
 12  num_decline                 1211571 non-null  int64  
 1

In [19]:
new_data.head()

,id,quality_sale,accepted_flg,seven_day_churn,thirty_day_churn,ninety_day_churn,clndr_dt_cst,start_dt,lines,previous_tech_support_call,...,latitude,longitude,month,hh_income_code,day_of_week,time_of_day,zipcode_avg_sales,zipcode_neg_sales,zipcode_pos_sales,zipcode_total_sales
0,c90ce3eaafffd07a32cb9cf4d153aef6f524b29aeef447...,0,0,NaN,NaN,NaN,2022-03-21 00:00:00.000,2022-03-21 20:44:17.405,2,15,...,33.7669,-111.9182,march,1,Monday,20,0.041667,69.0,3.0,72.0
1,91aa94d54079b20e2a18b09c070d0304b3df9c7272e9fc...,0,0,NaN,NaN,NaN,2022-03-03 00:00:00.000,2022-03-03 21:16:27.713,1,10,...,38.3824,-75.6336,march,3,Thursday,21,0.105590,144.0,17.0,161.0
2,eb269842f94437bfd7e3a5a1f30d7b9a84a842afd8a153...,0,0,NaN,NaN,NaN,2022-03-16 00:00:00.000,2022-03-16 13:59:15.707,5,30,...,34.4466,-118.5374,march,1,Wednesday,13,0.058824,112.0,7.0,119.0
3,fc05f45119a8a4b830a3463bfc8a7113b2807c88e7d2da...,0,0,NaN,NaN,NaN,2022-03-01 00:00:00.000,2022-03-01 12:33:55.917,5,1,...,21.4228,-157.8115,march,1,Tuesday,12,0.084034,109.0,10.0,119.0
4,68943556bf63277816d843ffb129e23ffe60bc5b982603...,1,1,0.0,0.0,0.0,2022-03-01 00:00:00.000,2022-03-01 12:22:15.780,2,4,...,30.3125,-95.4527,march,4,Tuesday,12,0.151163,73.0,13.0,86.0


In [20]:
new_data.to_csv("fe_data/total_encoded_tabular.csv", index = False)